### [Introduction to SIFT (Scale-Invariant Feature Transform)](https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html)
해리스 코너 감지기는 이미지의 크기가 변경 될 때 충분하지 않습니다. Lowe는 스케일 불변의 특징을 찾기위한 획기적인 방법을 개발했으며 SIFT라 불린다.

#### Goal
In this chapter,

* We will learn about the concepts of SIFT algorithm
* We will learn to find SIFT Keypoints and Descriptors.

#### Theory
지난 몇 장에서는 Harris 등의 코너 감지기를 보았습니다. 이들은 회전 불변인데, 이는 이미지가 회전하더라도 동일한 코너를 찾을 수 있음을 의미합니다. 모서리가 회전 된 이미지에서도 모퉁이에 남아 있기 때문에 분명합니다. 하지만 스케일링은? 이미지의 크기를 조정하면 구석이 구석이 아닐 수도 있습니다. 예를 들어 아래의 간단한 이미지를 확인하십시오. 동일한 창에서 확대 / 축소 할 때 작은 창 내의 작은 이미지의 구석이 평평합니다. 해리스 코너는 스케일 불변 적이 아닙니다.

![](sift_scale_invariant.jpg)

그래서 2004 년 브리티시 컬럼비아 대학의 **D.Lowe** 는 **키포인트** 를 추출하고 설명자를 계산하는 **Scale-Invariant Keypoints**의 **Distinctive Image Features** 라는 새로운 알고리즘 인 SIFT (Scale Invariant Feature Transform)를 만들었습니다. * (이 논문은 이해하기 쉽고 SIFT에서 입수 할 수있는 최상의 자료로 간주되므로이 설명은이 논문의 간략한 요약입니다).

주로 SIFT 알고리즘에 관련된 네 가지 단계가 있습니다. 우리는 그들을 하나 하나 보게 될 것입니다.

1. Scale-space Extrema Detection  

위의 이미지에서 우리는 동일한 창을 사용하여 다른 규모의 키포인트를 감지 할 수 없다는 것을 알 수 있습니다. 작은 모퉁이에서도 OK입니다. 그러나 더 큰 모서리를 탐지하기 위해서는 더 큰 창문이 필요합니다. 이를 위해 스케일 공간 필터링이 사용됩니다. 그것에서, Laplacian of Gaussian은 다양한 \\(\sigma\\) 값을 가진 이미지에서 발견됩니다. LoG는 \\(\sigma\\)의 변화로 인해 다양한 크기의 얼룩을 검출하는 얼룩 검출기 역할을합니다. 즉, \\(\sigma\\)는 스케일링 매개 변수의 역할을합니다. 예를 들어, 위의 이미지에서 낮은 \\(\sigma\\)를 갖는 가우시안 커널은 작은 모서리에 대해 높은 값을 제공하는 반면 높은 \\(\sigma\\)를 갖는 가우시안 커널은 큰 모서리에 적합합니다. 그래서 우리는 \\((x,y,\sigma)\\) 값의 목록을 제공하는 스케일과 공간에서 로컬 최대 값을 찾을 수 있습니다. 즉, \\(\sigma\\) 스케일에서 \\((x, y)\\)에 잠재적 인 키포인트가 있음을 의미합니다.

그러나 이 LoG는 약간의 비용이 들기 때문에 SIFT 알고리즘은 LoG의 근사치 인 Gaussian의 차를 사용합니다. Gaussian의 차이는 두 개의 다른 \\(\sigma\\)가있는 이미지의 가우시안 블러 링의 차이로 얻어지며 \\(\sigma\\)와 \\(k\sigma\\)가되도록합니다. 이 과정은 Gaussian Pyramid에서 이미지의 다른 옥타브에 대해 수행됩니다. 그것은 아래 이미지에 표현되어 있습니다

![](sift_dog.jpg)

일단이 DoG가 발견되면, 스케일 및 공간에 대해 로컬 극한을 검색합니다. 예를 들어, 이미지의 한 픽셀은 다음 스케일의 9 픽셀과 이전 스케일의 9 픽셀과 마찬가지로 8 개의 이웃과 비교됩니다. 로컬 극한치 인 경우 잠재적 인 키포인트입니다. 이것은 기본적으로 키포인트가 그 스케일에서 가장 잘 표현된다는 것을 의미합니다. 아래 그림과 같습니다 :

![](sift_local_extrema.jpg)

다른 매개 변수와 관련하여이 논문은 옥타브 수 = 4, 눈금 수 = 5, 초기 \\(\sigma=1.6, k=\sqrt{2} \\) 등을 최적 값으로 요약 할 수있는 경험적 데이터를 제공한다.

Regarding different parameters, the paper gives some empirical data which can be summarized as, number of octaves = 4, number of scale levels = 5, initial \\(\sigma=1.6, k=\sqrt{2} \\) etc as optimal values.

2. Keypoint Localization

일단 잠재적인 키포인트 위치가 발견되고, 더 정확한 결과를 얻으려면 세련되게 해야 합니다. 그들은 극한의 더 정확한 위치를 얻기 위해 스케일 공간의 Taylor 시리즈 확장을 사용했으며,이 극한치의 강도가 임계 값 (용지 당 0.03)보다 작으면 거부됩니다. 이 임계 값은 OpenCV에서 **contrastThreshold**라고 합니다.

DoG는 모서리에 대한 응답이 높기 때문에 모서리도 제거해야 합니다. 이를 위해 해리스 코너 감지기와 유사한 개념이 사용됩니다. 그들은 주 곡률을 계산하기 위해 2x2 헤시안 행렬(H)을 사용했습니다. 우리는 해리스 코너 검출기로부터 모서리의 경우 하나의 고유 값이 다른 것보다 크다는 것을 알고 있습니다. 그래서 여기에서는 간단한 함수를 사용했습니다.

이 비율이 OpenCV에서 **edgeThreshold** 라고하는 임계 값보다 크면 해당 키포인트가 삭제됩니다. 그것은 종이로 10으로 주어진다.

따라서 저 대비 키포인트와 에지 키포인트를 제거하고 남아있는 것은 중요한 관심사입니다.

3. Orientation Assignment

이제 각 키포인트에 오리엔테이션을 지정하여 이미지 회전에 대한 불변성을 얻습니다. 이웃은 눈금에 따라 키포인트 위치 주변에서 촬영되고 그레디언트 크기 및 방향은 해당 영역에서 계산됩니다. 360도를 포함하는 36 개의 저장소가있는 방향 막대 그래프가 작성됩니다. (키포인트의 스케일의 1.5 배에 해당하는 가중치 가중치 순환 창과 가중치 가중치에 의해 가중치가 적용됩니다. 히스토그램에서 가장 높은 피크가 찍히고 80 %를 초과하는 피크도 방향을 계산하는 것으로 간주됩니다. 같은 위치와 규모이지만 방향이 다른 키포인트. 일치의 안정성에 기여합니다.

4. Keypoint Descriptor

이제 키포인트 설명자가 생성됩니다. 키포인트 주변의 16x16 이웃을 촬영합니다. 4x4 크기의 16 개 하위 블록으로 나뉩니다. 각 하위 블록에 대해 8 개의 빈 방향 막대 그래프가 작성됩니다. 따라서 총 128 개의 bin 값을 사용할 수 있습니다. 키포인트 설명자를 형성하는 벡터로 나타납니다. 이 외에도 조명 변화, 회전 등에 대한 견고성을 확보하기 위해 여러 가지 조치가 취해집니다.

5. Keypoint Matching

두 이미지 사이의 키포인트는 가장 가까운 이웃을 식별하여 일치 시킵니다. 그러나 경우에 따라 두 번째 가장 가까운 성냥은 첫 번째 성냥에 아주 가깝습니다. 소음 또는 다른 이유로 인해 발생할 수 있습니다. 이 경우 가장 가까운 거리와 가장 가까운 거리의 비율이 사용됩니다. 0.8보다 크면 거부됩니다. 그것은 종이 당 3 %의 정확한 매치만을 버리고 거짓 매치의 약 90 %를 제거합니다.

그래서 이것은 SIFT 알고리즘의 요약입니다. 자세한 내용과 이해를 위해 원본 종이를 읽는 것이 좋습니다. 한가지 기억하십시오,이 알고리즘은 특허입니다. 따라서이 알고리즘은 opencv contrib repo에 포함되어 있습니다 .

#### SIFT in OpenCV
So now let's see SIFT functionalities available in OpenCV. Let's start with keypoint detection and draw them. First we have to construct a SIFT object. We can pass different parameters to it which are optional and they are well explained in docs.


In [1]:
import numpy as np
import cv2 as cv
import bonghanUtil as util

img  = cv.imread('home.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

sift = cv.xfeatures2d.SIFT_create()
kp   = sift.detect(gray, None)
img  = cv.drawKeypoints(gray, kp, img)

cv.imwrite('sift_keypoints.jpg',img)
util.showImage(img)

In [2]:
%%writefile 'TestCode.py'
import numpy as np
import cv2 as cv
import bonghanUtil as util

img  = cv.imread('home.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

sift = cv.xfeatures2d.SIFT_create()
kp   = sift.detect(gray, None)
img  = cv.drawKeypoints(gray, kp, img)

cv.imwrite('sift_keypoints.jpg',img)
util.showImage(img)

Overwriting 'TestCode.py'


sift.detect () 함수는 이미지에서 키포인트를 찾습니다. 이미지의 일부만 검색하려면 마스크를 전달할 수 있습니다. 각 키포인트는 (x, y) 좌표, 의미있는 이웃의 크기, 방향을 지정하는 각도, 키포인트의 강도를 지정하는 응답 등과 같은 많은 특성을 갖는 특수 구조입니다.

OpenCV는 키포인트의 위치에 작은 원을 그려 주는 cv.drawKeyPoints () 함수 도 제공합니다 . 플래그에 cv.DRAW_MATCHES_FLAGS_FLAWS_DRAW_RICH_KEYPOINTS 를 건네 주면 (자), 키포인트의 사이즈로 원을 묘화 해, 그 방향을 나타냅니다. 아래 예제를 참조하십시오.

In [3]:
img=cv.drawKeypoints(gray,kp,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imwrite('sift_keypoints.jpg',img)
util.showImage(img)

아래의 두 가지 결과를 참조하십시오.
![](sift_keypoints_res.jpg)

이제 기술자를 계산하기 위해 OpenCV는 두 가지 방법을 제공합니다.

이미 키포인트를 발견 했으므로 sift.compute () 를 호출 하여 발견 한 키포인트로부터 디스크립터를 계산할 수 있습니다 . 예 : kp, des = sift.compute (회색, kp)
키포인트를 찾지 못한 경우 sift.detectAndCompute () 함수를 사용하여 키포인트와 설명자를 한 번에 직접 찾을 수 있습니다.
두 번째 방법은 다음과 같습니다.


In [4]:
import numpy as np
import cv2 as cv
import bonghanUtil as util

img  = cv.imread('home.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

sift    = cv.xfeatures2d.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)
img  = cv.drawKeypoints(gray, kp, img)

cv.imwrite('sift_keypoints.jpg',img)
util.showImage(img)

Here kp will be a list of keypoints and des is a numpy array of shape \\(Number\_of\_Keypoints \times 128 \\)

So we got keypoints, descriptors etc. Now we want to see how to match keypoints in different images. That we will learn in coming chapters.

#### Additional Resources
#### Exercises